In [20]:
from pathlib import Path
import yaml
from fastrelease.conda import latest_pypi, pypi_json
from fastcore.all import ifnone, patch, store_attr , compose

In [21]:
@patch
def wlns(self:Path, lst:list): 
    with self.open('w') as f: f.writelines(lst)

@patch
def d2yml(self:Path, d):
    yaml.SafeDumper.ignore_aliases = lambda *args : True
    with self.open('w') as f: yaml.safe_dump(d, f)
        
def _mkdir(path):
    p = Path(path)
    p.mkdir(exist_ok=True, parents=True)
    return p

In [22]:
class CondaBuild:
    def __init__(self, pypinm, import_nm, deps=[], path=None):
        store_attr('pypinm,import_nm,deps')
        self.ver = str(latest_pypi(pypinm))
        self.info = pypi_json(f'{pypinm}/{self.ver}')['info']
        self.path = _mkdir(ifnone(path,self.pypinm))
        self.meta ={'package': {'name': self.pypinm, 'version': self.ver},
                    'build':{'number':0, 'binary_relocation':False, 'detect_binary_files_with_prefix':False},
                    'requirents':{'host': ['pip', 'python'], 'run':self.deps},
                    'test':{'imports': [self.import_nm], 'requires':['pip']},
                    'about':{'home':self.info['home_page'], 'summary':self.info['summary'], 'license':self.info['license']},
                    'extra':{'recipe-maintainers': ['jph00']}
                   }
        
    def create_meta(self): (self.path/'meta.yaml').d2yml(self.meta)
            
    def create_sh(self): (self.path/'build.sh').wlns(['#!/usr/bin/env bash\n','PIP_NO_INDEX=False python -m pip install -Uq $PKG_NAME'])
            
    def create_bat(self): (self.path/'bld.bat').wlns(['setlocal\n', 'set PIP_NO_INDEX=False\n','python -m pip install -Uq %PKG_NAME%'])
    
    def create_build_files(self): 
        for f in [self.create_meta, self.create_sh, self.create_bat]: f()
        

In [23]:
cb = CondaBuild('opencv-python-headless', import_nm='cv2', path='test')
cb.create_build_files()